## Example 2 - Meshes for Topography 

This notebook introduces the `TopoMesh` object, which builds the following data structures:

- hill slope
- downhill propagation matrices
- upstream area

in addition to the data structures inherited from `FlatMesh`. These form the necessary structures to propagate information from higher to lower elevations. Derivatives are computed on the mesh to calculate the height field, smoothing operators are available to reduce short wavelength features and artefacts.

In this notebook we setup a height field and calculate its derivatives on an unstructued mesh. We smooth the derivatives using the radial-basis function (RBF) smoothing kernel.

> Note: The API for the structured mesh is identical

#### Contents

- [Height field](#Height-field)
- [Derivatives and hill slope](#Derivatives-and-hill-slope)
- [Upstream area and stream power](#Upstream-area-and-stream-power)
- [Outflow analysis](#Outflow analysis)

In [1]:
from quagmire.tools import meshtools
from quagmire import FlatMesh, TopoMesh, SurfaceProcessMesh

In [2]:
import matplotlib.pyplot as plt
import numpy as np
# from scipy.ndimage import imread
# from quagmire import tools as meshtools
# from quagmire import SurfaceProcessMesh
%matplotlib inline

In [3]:
minX, maxX = -5.0, 5.0
minY, maxY = -5.0, 5.0,
dx, dy = 0.02, 0.02

x, y, simplices = meshtools.elliptical_mesh(minX, maxX, minY, maxY, dx, dy)

DM = meshtools.create_DMPlex_from_points(x, y, bmask=None)

In [4]:
mesh = TopoMesh(DM, downhill_neighbours=1)

print ("Triangulation has {} points".format(mesh.npoints))

0 - Delaunay triangulation 0.383572s
0 - Calculate node weights and area 0.00742699999999985s
0 - Find boundaries 0.00033199999999933283s
0 - cKDTree 0.009533999999999487s
0 - Construct neighbour cloud array 0.3461350000000003s
0 - Construct rbf weights 0.07085700000000017s
Triangulation has 62234 points


## Height field

We generate a cylindrically symmetry domed surface and add multiple channels incised along the boundary. The height and slope fields reside as attributes on the `SurfaceProcessMesh` instance.

In [ ]:
radius  = np.sqrt((x**2 + y**2))
theta   = np.arctan2(y,x) + 0.1

height  = np.exp(-0.025*(x**2 + y**2)**2) + 0.25 * (0.2*radius)**4  * np.cos(5.0*theta)**2 ## Less so
height  += 0.5 * (1.0-0.2*radius)
height  += np.random.random(height.size) * 0.01 # random noise

In [ ]:
mesh.update_height(height)


## Derivatives and hill slopes

**Derivatives** can be evaluated on the mesh using the inbuilt routine in the `stripy` object. It employs automatically selected tension factors to preserve shape properties of the data and avoid overshoot and undershoot associated with steep gradients. **Note:** In parallel it is wise to check if this tensioning introduces artefacts near the boundaries.

```python
dfdx, dfdy = mesh.derivative_grad(f, nit=10, tol=1e-8):
```

where `nit` and `tol` control the convergence criteria.

---

The **radial-basis function** (RBF) smoothing kernel works by setting up a series of gaussian functions based on the distance $d$ between neighbouring nodes and a `delta` scaling factor, $\Delta$:

$$
W_i = \frac{\exp \left( \frac{d_i}{\Delta} \right)^2}{\sum_{i} \left( \frac{d_i}{\Delta} \right)^2}
$$

`delta` is set to the mean distance between nodes by default, but it may be changed to increase or decrease the _smoothness_:

```python
smoothed_field = mesh.rbf_smoother(field, iterations=1, delta=None)
```

In [ ]:
def smooth_slope(mesh, height, delta=None):
    """ Compute the slope using the RBF smoothing kernel """
    
    mesh._construct_rbf_weights(delta)
    rbf_height = mesh.rbf_smoother(height)
    rbf_gradx, rbf_grady = mesh.derivative_grad(rbf_height)
    return np.hypot(rbf_gradx, rbf_grady)


# analytic slope

rbf_slope1 = smooth_slope(mesh, mesh.heightVariable.data, delta=0.01)
rbf_slope2 = smooth_slope(mesh, mesh.heightVariable.data, delta=0.03)
rbf_slope3 = smooth_slope(mesh, mesh.heightVariable.data, delta=0.05)


In [ ]:
import lavavu

points = np.column_stack([mesh.tri.points, height])

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

tri1 = lv.triangles("triangles")
tri1.vertices(points)
tri1.indices(mesh.tri.simplices)
tri1.values(mesh.slopeVariable.data, "slope")
tri1.values(rbf_slope1, "smooth_slope1")
tri1.values(rbf_slope2, "smooth_slope2")
tri1.values(rbf_slope3, "smooth_slope3")

tri1.colourmap("#990000 #FFFFFF #000099")
tri1.colourbar()

lv.control.Panel()
lv.control.ObjectList()
tri1.control.List(["slope", "smooth_slope1", "smooth_slope2", "smooth_slope3", ], property="colourby", value="orginal", command="redraw")
lv.control.show()

We can smooth the topography itself (not just the slope) 




In [ ]:
mesh._construct_rbf_weights(mesh.neighbour_cloud_distances.mean()*0.5)
smoothed_height = mesh.rbf_smoother(mesh.heightVariable.data)

mesh1 = SurfaceProcessMesh(DM)
mesh1.update_height(smoothed_height)


In [ ]:
points = np.column_stack([mesh1.tri.points, mesh1.heightVariable.data])

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

tri1 = lv.triangles("triangles")
tri1.vertices(points)
tri1.indices(mesh1.tri.simplices)
tri1.values(mesh1.slopeVariable.data, "slope")
tri1.values(mesh1.heightVariable.data-mesh.heightVariable.data, "deltah")
tri1.values(mesh1.slopeVariable.data-mesh.slopeVariable.data, "delta slope")

tri1.colourmap("#990000 #FFFFFF #000099")
tri1.colourbar()

lv.control.Panel()
lv.control.ObjectList()
tri1.control.List(["slope", "deltah", "delta slope" ], property="colourby", value="orginal", command="redraw")
lv.control.show()